<a href="https://colab.research.google.com/github/tpmmthomas/Sepsis-diagnosis-from-pairwise-single-cell-RNA-Continued/blob/master/NEW_NEW_capsnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D, BatchNormalization
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from tensorflow import keras
from sklearn.metrics import *
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from PIL import Image
from tensorflow.keras import initializers, layers
from datetime import date
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.utils.multi_gpu_utils import multi_gpu_model
from tensorflow.python.client import device_lib

2022-03-10 19:55:54.814796: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [34]:
class Length(layers.Layer):
    def call(self, inputs, **kwargs):
        return K.sqrt(K.sum(K.square(inputs), -1))

    def compute_output_shape(self, input_shape):
        return input_shape[:-1]


class Mask(layers.Layer):
    def call(self, inputs, **kwargs):
        if type(inputs) is list:  # true label is provided with shape = [None, n_classes], i.e. one-hot code.
            assert len(inputs) == 2
            inputs, mask = inputs
        else:  # if no true label, mask by the max length of capsules. Mainly used for prediction
            # compute lengths of capsules
            x = K.sqrt(K.sum(K.square(inputs), -1))
            # generate the mask which is a one-hot code.
            # mask.shape=[None, n_classes]=[None, num_capsule]
            mask = K.one_hot(indices=K.argmax(x, 1), num_classes=tf.shape(x)[1])
        masked = K.batch_flatten(inputs * K.expand_dims(mask, -1))
        return masked

    def compute_output_shape(self, input_shape):
        if type(input_shape[0]) is tuple:  # true label provided
            return tuple([None, input_shape[0][1] * input_shape[0][2]])
        else:  # no true label provided
            return tuple([None, input_shape[1] * input_shape[2]])


def squash(vectors, axis=-1):
    s_squared_norm = K.sum(K.square(vectors), axis, keepdims=True)
    scale = s_squared_norm / (1 + s_squared_norm) / K.sqrt(s_squared_norm + K.epsilon())
    return scale * vectors


class CapsuleLayer(layers.Layer):
    def __init__(self, num_capsule, dim_capsule, routings=3,
                 kernel_initializer='glorot_uniform',
                 **kwargs):
        super(CapsuleLayer, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.kernel_initializer = initializers.get(kernel_initializer)

    def build(self, input_shape):
        assert len(input_shape) >= 3, "The input Tensor should have shape=[None, input_num_capsule, input_dim_capsule]"
        self.input_num_capsule = input_shape[1]
        self.input_dim_capsule = input_shape[2]

        # Transform matrix
        self.W = self.add_weight(shape=(self.num_capsule, self.input_num_capsule,
                                        self.dim_capsule, self.input_dim_capsule),
                                 initializer=self.kernel_initializer,
                                 name='W')

        self.built = True

    def call(self, inputs, training=None):

      inputs_expand = tf.expand_dims(inputs, 1)
      inputs_tiled  = tf.tile(inputs_expand, [1, self.num_capsule, 1, 1])
      inputs_tiled  = tf.expand_dims(inputs_tiled, 4)
      inputs_hat = tf.map_fn(lambda x: tf.matmul(self.W, x), elems=inputs_tiled)     

      # Begin: Routing algorithm ----------------------------------------------#
      b = tf.zeros(shape=[tf.shape(inputs_hat)[0], self.num_capsule, 
                          self.input_num_capsule, 1, 1])

      assert self.routings > 0, 'The routings should be > 0.'
      for i in range(self.routings):
        c = layers.Softmax(axis=1)(b)
        outputs = tf.multiply(c, inputs_hat)
        outputs = tf.reduce_sum(outputs, axis=2, keepdims=True)
        outputs = squash(outputs, axis=-2)  # [None, 10, 1, 16, 1]

        if i < self.routings - 1:
          outputs_tiled = tf.tile(outputs, [1, 1, self.input_num_capsule, 1, 1])
          agreement = tf.matmul(inputs_hat, outputs_tiled, transpose_a=True)
          b = tf.add(b, agreement)

      # End: Routing algorithm ------------------------------------------------#
      outputs = tf.squeeze(outputs, [2, 4])
      return outputs,c

    def compute_output_shape(self, input_shape):
        return tuple([None, self.num_capsule, self.dim_capsule])

class Weightlayer(layers.Layer):
    def __init__(self, dim_capsule, n_channels, **kwargs):
        super(Weightlayer, self).__init__(**kwargs)
        self.filters = dim_capsule*n_channels
        
    def build(self, input_shape):
        self.weight = self.add_weight(shape=(1,input_shape[1],self.filters), initializer='glorot_uniform', name='weight')
        self.built = True

    def call(self, inputs):
        return tf.multiply(inputs , self.weight) , self.weight   
    
def PrimaryCap(inputs, dim_capsule, n_channels, kernel_size, strides, padding):
    inputs, layerweight = Weightlayer(dim_capsule, n_channels)(inputs)
    outputs = layers.Reshape(target_shape=[-1, dim_capsule], name='primarycap_reshape')(inputs)
    return layers.Lambda(squash, name='primarycap_squash')(outputs)

In [3]:
xtrain = r"./dataSC/NEW_training_sample.csv.gz"
ytrain = r"./dataSC/NEW_training_label.csv.gz"
xtest = r"./dataSC/NEW_testing_sample.csv.gz"
ytest = r"./dataSC/NEW_testing_label.csv.gz"

i=0
samplesdf = pd.read_csv(xtrain,compression ="gzip",delimiter=',')
x_train = samplesdf.to_numpy()

samplesdf = pd.read_csv(ytrain,compression ="gzip",delimiter=',')
y_train = samplesdf.to_numpy()

samplesdf = pd.read_csv(xtest,compression ="gzip",delimiter=',')
x_test = samplesdf.to_numpy()

samplesdf = pd.read_csv(ytest,compression ="gzip",delimiter=',')
y_test = samplesdf.to_numpy()

print("done")
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

done
(38483, 3030)
(38483, 1)
(4265, 3030)
(4265, 1)


In [4]:
common_indicator = pd.read_csv('./dataBulk/common_rna_indicator.csv',index_col=0).values.squeeze().tolist()

x_train = x_train.T[common_indicator].T
print(x_train.shape)
x_test = x_test.T[common_indicator].T
print(x_test.shape)

(38483, 2869)
(4265, 2869)


In [5]:
# train, validation, test ratio = 80%:10%:10%
xtr,xval, ytr,yval = train_test_split(x_train,y_train,test_size=4265,random_state=42)
print(xtr.shape,ytr.shape)
print(xval.shape,yval.shape)

(34218, 2869) (34218, 1)
(4265, 2869) (4265, 1)


In [6]:
xtr=tf.reshape(xtr,(len(xtr),2869,1))
xval=tf.reshape(xval,(len(xval),2869,1))
x_test=tf.reshape(x_test,(len(x_test),2869,1))
print(xtr.shape)
print(xval.shape)
print(x_test.shape)

2022-03-10 19:56:34.904304: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-10 19:56:34.905438: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.5/lib64
2022-03-10 19:56:34.905473: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-10 19:56:34.905541: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (hpc1.cse.cuhk.edu.hk): /proc/driver/nvidia/version does not exist
2022-03-10 19:56:34.906876: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2

(34218, 2869, 1)
(4265, 2869, 1)
(4265, 2869, 1)


In [28]:
#Transformer Structure
class MultiHeadSelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads=8):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        if embed_dim % num_heads != 0:
            raise ValueError(
                f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}"
            )
        self.projection_dim = embed_dim // num_heads
        self.query_dense = layers.Dense(embed_dim)
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        self.combine_heads = layers.Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)  # (batch_size, seq_len, embed_dim)
        key = self.key_dense(inputs)  # (batch_size, seq_len, embed_dim)
        value = self.value_dense(inputs)  # (batch_size, seq_len, embed_dim)
        query = self.separate_heads(
            query, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        key = self.separate_heads(
            key, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        value = self.separate_heads(
            value, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(
            attention, perm=[0, 2, 1, 3]
        )  # (batch_size, seq_len, num_heads, projection_dim)
        concat_attention = tf.reshape(
            attention, (batch_size, -1, self.embed_dim)
        )  # (batch_size, seq_len, embed_dim)
        output = self.combine_heads(
            concat_attention
        )  # (batch_size, seq_len, embed_dim)
        return output

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadSelfAttention(embed_dim, num_heads)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [35]:
def CapsNet(input_shape, n_class, routings):
    inputs = layers.Input(shape=input_shape)
    primarycaps = PrimaryCap(inputs, dim_capsule=8, n_channels=1, kernel_size=1, strides=1, padding='valid')
    groupcaps,routing_weight = CapsuleLayer(num_capsule=n_class, dim_capsule=16, routings=routings,
                             name='groupcaps')(primarycaps)

    transformer_block = TransformerBlock(embed_dim=16, num_heads=2, ff_dim=128)
    x = transformer_block(groupcaps)
    x = layers.Flatten()(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(150, activation="relu")(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = models.Model(inputs, outputs)
    return model


def margin_loss(y_true, y_pred):
    L = y_true * K.square(K.maximum(0., 0.9 - y_pred)) + \
        0.5 * (1 - y_true) * K.square(K.maximum(0., y_pred - 0.1))
    return K.mean(K.sum(L, 1))


def test(model, X_test, y_test, args):
    y_pred, x_recon = model.predict(X_test, batch_size=20)
    print('-'*30 + 'Begin: test' + '-'*30)
    print('Test acc:', np.sum(np.argmax(y_pred, 1) == np.argmax(y_test, 1))/y_test.shape[0])
    return y_pred

In [36]:
 # define model
model= CapsNet(input_shape=xtr[1,:,:].shape,n_class=20,routings=3)
model.summary()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 2869, 1)]         0         
_________________________________________________________________
weightlayer_11 (Weightlayer) ((None, 2869, 8), (1, 286 22952     
_________________________________________________________________
primarycap_reshape (Reshape) (None, 2869, 8)           0         
_________________________________________________________________
primarycap_squash (Lambda)   (None, 2869, 8)           0         
_________________________________________________________________
groupcaps (CapsuleLayer

In [10]:
# train or test
# compile the model
model.compile(optimizer=optimizers.Adam(lr=0.0001, amsgrad=True),loss='binary_crossentropy', metrics=["accuracy"])

from tensorflow.keras import callbacks
earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", 
                                        mode ="min", patience = 5, 
                                        restore_best_weights = True)
history = model.fit(xtr, ytr, batch_size=32, epochs=20,validation_data=(xval, yval),
               shuffle=True)

today = date.today()
model.save('./model/weightmodel')

Epoch 1/2


2022-03-10 19:56:36.941583: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-03-10 19:56:36.942515: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1895295000 Hz


1070/1070 [==============================] - 907s 843ms/step - loss: 0.5811 - accuracy: 0.6894 - val_loss: 0.4533 - val_accuracy: 0.7798
Epoch 2/2
1070/1070 [==============================] - 791s 740ms/step - loss: 0.3998 - accuracy: 0.8134 - val_loss: 0.4022 - val_accuracy: 0.8106


2022-03-10 20:24:57.496118: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./model/1Convmodeltest/assets


INFO:tensorflow:Assets written to: ./model/1Convmodeltest/assets


In [ ]:
testresult=model.predict(x_test)
i = 0
correct = 0
for x in testresult:
    if x >=0.5 and y_test[i] == 1:
        correct = correct + 1
    elif x < 0.5 and y_test[i] == 0:
        correct = correct + 1
    i = i + 1
testacc = correct/i
print("Testing accuracy:",testacc)

fpr, tpr, _ = roc_curve(y_test,testresult)
roc_auc = auc(fpr,tpr)
print("AUROC = %.02f"% roc_auc)

precision, recall, _ = precision_recall_curve(y_test,testresult)
prc_auc = auc(recall,precision)
print("AUPRC = %.02f"% prc_auc)
ss = np.zeros((len(testresult)))
i = 0
for x in testresult:
    if x >= 0.5:
        ss[i] = 1
    else:
        ss[i] = 0
    i = i + 1
f1s = f1_score(y_test,ss)
print("f1_score = %.02f"% f1s)

In [ ]:
pd.DataFrame(testresult).to_csv('./biomarkers/capsnetweighted.csv',header=None, index=None)
pd.DataFrame(y_test).to_csv('./biomarkers/capsnet_y.csv',header=None, index=None)